<h3>data/temperature/*.csv -> result/temperature/result.csv</h3>
<ol style="color: blue">
  <li>Remove empty lines </li>
  <li>Replace date to year </li>
  <li>Remove cells with older than 1990</li>
  <li>Groups by country+year and sum averages for temperatures</li>
</ol>

In [ ]:
! cat data/temperature/*.csv | awk -f com/temperature/removeEmpty.awk | awk -f com/temperature/replaceYearDate.awk | awk -f com/temperature/filterYears.awk | awk -f com/temperature/groupByYearAverageSum.awk  > result/temperature/result.csv

In [ ]:
<ol style="color: green">
  <li>Remove empty lines </li>
  <li>Replace date to year </li>
  <li>Remove cells with older than 1990</li>
  <li>Groups by country+year and sum averages for temperatures</li>
</ol>

In [41]:
! awk -f com/energy/merge.awk data/energy/*.csv | awk -f com/energy/filterYears.awk  > result/energy/merged.csv